# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである．体系的にまとまってはいない．

## 1. 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける．

関数内で配列を変更する場合には注意が必要である．以下に入力された配列を同じサイズの要素1の配列で置き換える，ということを目的として書かれた2つの関数がある．違いは`v`の後に`[:]`としているかどうかである．

In [1]:
function wrong!(a::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 1 method)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる．

In [2]:
using Random
v = rand(2, 2)
println("v : ", v)

wrong!(v)
println("wrong : ", v)

right!(v)
println("right : ", v)

v : 

[0.8151886759340627 0.12223274551874708; 0.24913419841648077 0.38480789913659885]
wrong : [0.8151886759340627 0.12223274551874708; 0.24913419841648077 0.38480789913659885]
right : [1.0 1.0; 1.0 1.0]


## 2. 配列の1次元化
配列を一次元化(flatten)する方法．まずは3次元配列を作成する．

In [3]:
B = rand(2, 2, 2)

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.179436  0.657939
 0.753243  0.897187

[:, :, 2] =
 0.878666  0.25904
 0.126811  0.362235

用意されている`flatten`を素直に用いると次のようになる．

In [4]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Vector{Float64}:
 0.17943580226620193
 0.7532431329637357
 0.6579394666468243
 0.8971868179666216
 0.8786664348993651
 0.12681084982514057
 0.25904019442817283
 0.3622345396720148

ただし，単に`B[:]`とするだけでもよい．

In [5]:
B[:]

8-element Vector{Float64}:
 0.17943580226620193
 0.7532431329637357
 0.6579394666468243
 0.8971868179666216
 0.8786664348993651
 0.12681084982514057
 0.25904019442817283
 0.3622345396720148

## 3. 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある．これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある．以下では行ごとの和を1にする．

In [6]:
W = rand(3,3)

3×3 Matrix{Float64}:
 0.492264  0.566839   0.254865
 0.436766  0.0974801  0.124991
 0.953775  0.707431   0.578255

In [7]:
Wnormed = W ./ sum(W, dims=1)

3×3 Matrix{Float64}:
 0.261452  0.413223   0.266008
 0.231976  0.0710625  0.130456
 0.506572  0.515714   0.603536

In [8]:
println(sum(Wnormed, dims=1))

[1.0 1.0 1.0]


## 4. 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる．まず，2つの行列A, Bを用意する．

In [9]:
A = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [10]:
B = [4 5 6; 7 8 9]

2×3 Matrix{Int64}:
 4  5  6
 7  8  9

### 4.1 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と，`[ ]`を使うやり方がある．

In [11]:
H1 = hcat(A,B)

2×5 Matrix{Int64}:
 1  2  4  5  6
 3  4  7  8  9

In [12]:
H2 = [A B]

2×5 Matrix{Int64}:
 1  2  4  5  6
 3  4  7  8  9

なお，MATLABのように次のようにすると正しく結合はされない．

In [13]:
H3 = [A, B]

2-element Vector{Matrix{Int64}}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 4.2 垂直結合 (Vertical concatenation)

In [14]:
V1 = vcat(A, B')

5×2 Matrix{Int64}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [15]:
V2 = [A; B']

5×2 Matrix{Int64}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [16]:
[V2 [A;B']]

5×4 Matrix{Int64}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9

## 5. 配列に新しい軸を追加
要はnumpyでの`A[None, :]`や`A[np.newaxis, :]`のようなことがしたい場合．やや面倒だが，`reshape`を使うか，`[CartesianIndex()]`を用いる．

In [17]:
v = rand(3)

3-element Vector{Float64}:
 0.5639587438508948
 0.9483737965772752
 0.014513877145525766

In [18]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Matrix{Float64}:
 0.563959  0.948374  0.0145139

## 6. Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると，1次元配列のリストを2次元配列に変換できた．Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる．

In [19]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : 

Vector

{Vector{Float64}}
Size : (5,)


In [20]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : LinearAlgebra.Adjoint{Float64, Matrix{Float64}}
Size : (5, 3)


以下は多次元配列の場合．`cat(...)`で配列を結合し，`permitedims`で転置する．

In [21]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Vector{Array{Float64, 3}}
Size : (6,)


In [22]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64, 4}
Size : (6, 3, 4, 5)


## 7. 二項分布 (bernoulli distribution)のサンプリング

In [23]:
p = 0.7
N = 100

100

In [24]:
using BenchmarkTools

In [25]:
@benchmark floor.(p .+ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.91 KiB
  allocs estimate:  7
  --------------
  minimum time:     856.716 ns (0.00% GC)
  median time:      885.075 ns (0.00% GC)
  mean time:        944.270 ns (2.54% GC)
  maximum time:     11.234 μs (90.16% GC)
  --------------
  samples:          10000
  evals/sample:     67

In [26]:
println(sum(floor.(p .+ rand(N)) .== 1.0) / N) 

0.76


In [27]:
@benchmark 1.0f0 * (p .≥ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.55 KiB
  allocs estimate:  6
  --------------
  minimum time:     907.143 ns (0.00% GC)
  median time:      957.143 ns (0.00% GC)
  mean time:        1.033 μs (2.35% GC)
  maximum time:     31.418 μs (95.28% GC)
  --------------
  samples:          10000
  evals/sample:     28

In [28]:
println(sum(1.0f0 * (p .≥ rand(N)) .== 1.0) / N) 

0.74
